In [1]:
import numpy as np
import uproot
from copy import deepcopy
import os
import glob
import plotly.graph_objs as go
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, Output
from IPython.display import display, clear_output
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.optimize import fmin
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
df = uproot.open("../build/test.root:Hits")["pixelHits"].arrays(library="pd")
geom = uproot.open("../build/test.root:geometry").arrays(library="np")

In [17]:
print(df.iloc[0])
row = df.iloc[0]
# long_df = pd.DataFrame({
#     "x": row["hit_rowID"],
#     "y": row["hit_colID"],
#     "z": row["hit_layerID"],
#     "pdg": row["hit_pdgc"]
# })

long_df = pd.DataFrame({
    "x": [geom['pixel_xpos'][0][int(i)] for i in row["hit_rowID"]],
    "y": [geom['pixel_Ypos'][0][int(i)] for i in row["hit_colID"]],
    "z": [geom['pixel_Zpos'][0][int(i)] for i in row["hit_layerID"]],
    "pdg": row["hit_pdgc"]
})

print(set(long_df["pdg"]))

event_id                                                       0
hit_rowID      [5257.0, 5258.0, 5233.0, 5233.0, 5208.0, 5184....
hit_colID      [4320.0, 4320.0, 4319.0, 4318.0, 4318.0, 4318....
hit_layerID    [0.0, 0.0, 1.0, 1.0, 2.0, 3.0, 3.0, 4.0, 5.0, ...
hit_pdgc       [13, 13, 13, 11, 13, 13, -11, 13, 13, 13, 13, ...
hit_trackID    [1, 1, 1, 8, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1,...
hit_px         [-14854.884, -14854.927, -14854.588, -1.433570...
hit_py         [-458.92657, -458.9385, -482.2431, -3.1440194,...
hit_pz         [290828.53, 290828.53, 290799.28, 13.838646, 2...
hit_energy     [291208.03, 291208.03, 291178.84, 14.272674, 2...
hit_charge     [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0...
Name: 0, dtype: object
{-11, 11, 13}


In [18]:
def pdg_to_particle(pdg):
    if abs(pdg) == 11:
        return "electron"
    if abs(pdg) == 13:
        return "muon"
    return "other"

long_df["particle"] = long_df["pdg"].apply(pdg_to_particle)


In [19]:
import plotly.express as px

fig = px.scatter_3d(
    long_df,
    x="x",
    y="y",
    z="z",
    color="particle",
    color_discrete_map={
        "electron": "blue",
        "muon": "orange",
        "other": "gray"
    },
    opacity=0.8
)

fig.update_traces(marker=dict(size=4))
fig.show()
